### 线性判别分析
Linear Discriminant Analysis
LDA降维

In [1]:
import pandas as pd


In [2]:
feature_dict = {i:label for i,label in zip(range(4),('sepal length in cm',
                                                     'sepal width in cm',
                                                     'petal length in cm',
                                                     'petal width in cm'))}

In [11]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context # 全局取消证书验证

# df = pd.io.parsers.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
#                             header=None, sep=',')
df = pd.read_csv('/Users/joey/Documents/PycharmProjects/mywork/dimensionality_reduction_algorithm/iris.data',
                 header=None, sep=',')
df.columns = [l for i,l in sorted(feature_dict.items())] + ['class label']
df.dropna(how='all', inplace=True)
df.head()

,sepal length in cm,sepal width in cm,petal length in cm,petal width in cm,class label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [14]:
# 标签转换
from sklearn.preprocessing import LabelEncoder

X = df[['sepal length in cm','sepal width in cm','petal length in cm','petal width in cm']].values
y = df['class label'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) +1
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [15]:
type(y)

numpy.ndarray

### 每类的均值
分别求三种鸢尾花数据在不同特征维度上的均值向量 mi


In [16]:
import numpy as np
np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,4):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
    print('Mean Vector class %s: %s\n' % (cl, mean_vectors[cl-1]))


Mean Vector class 1: [5.006 3.418 1.464 0.244]

Mean Vector class 2: [5.936 2.77  4.26  1.326]

Mean Vector class 3: [6.588 2.974 5.552 2.026]



### 计算两个 4×4 维矩阵：类内散布矩阵和类间散布矩阵

In [17]:
S_W = np.zeros((4, 4))
for cl, mv in zip(range(1, 4), mean_vectors):
    class_sc_mat = np.zeros((4, 4))
    for row in X[y==cl]:
        row, mv = row.reshape(4, 1), mv.reshape(4, 1)
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat
print('within_class Scatter Matrix:\n',S_W)



within_class Scatter Matrix:
 [[38.9562 13.683  24.614   5.6556]
 [13.683  17.035   8.12    4.9132]
 [24.614   8.12   27.22    6.2536]
 [ 5.6556  4.9132  6.2536  6.1756]]


### 类间散布矩阵

In [19]:
overall_mean = np.mean(X, axis=0)
overall_mean

array([5.8433, 3.054 , 3.7587, 1.1987])

In [27]:
S_B = np.zeros((4, 4))
for i, mean_vec in enumerate(mean_vectors):
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(4, 1)
    overall_mean = overall_mean.reshape(4, 1)
    S_B = n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

print('between-class Scatter Matrix:\n', S_B)

between-class Scatter Matrix:
 [[ 27.7264  -2.9787  66.7718  30.8044]
 [ -2.9787   0.32    -7.1733  -3.3093]
 [ 66.7718  -7.1733 160.8022  74.1842]
 [ 30.8044  -3.3093  74.1842  34.224 ]]


求解矩阵特征值

In [35]:
# 矩阵的逆 和 求解特征值
# eig：矩阵特征
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

for i in range(len(eig_vals)):
    eigvec_sc = eig_vecs[:,i].reshape(4, 1)
    print('\nEigenvector {} \n{}'.format(i+1, eigvec_sc.real))
    print('\nEigenvector {:} \n{:.2e}'.format(i+1, eig_vals[i].real))


eig_vals, eig_vecs


Eigenvector 1 
[[-0.9084]
 [-0.1427]
 [ 0.2208]
 [ 0.3252]]

Eigenvector 1 
0.00e+00

Eigenvector 2 
[[-0.2007]
 [-0.3343]
 [ 0.5172]
 [ 0.7618]]

Eigenvector 2 
1.15e+01

Eigenvector 3 
[[ 0.1138]
 [ 0.9894]
 [ 0.0351]
 [-0.0827]]

Eigenvector 3 
-2.74e-16

Eigenvector 4 
[[ 0.3479]
 [-0.8682]
 [-0.2819]
 [ 0.2138]]

Eigenvector 4 
9.90e-16


(array([ 0.0000e+00,  1.1502e+01, -2.7445e-16,  9.9015e-16]),
 array([[-0.9084, -0.2007,  0.1138,  0.3479],
        [-0.1427, -0.3343,  0.9894, -0.8682],
        [ 0.2208,  0.5172,  0.0351, -0.2819],
        [ 0.3252,  0.7618, -0.0827,  0.2138]]))

### 特征值与特征向量：
- 特征向量：表示映射方向
- 特征值：特征向量的重要程度